In [1]:
import os
import pickle
import sqlite3
import shutil

import cv2
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

Extract the Photo table from the database

In [2]:
BCU_database_folder = '../data/BCU_database/'
dataset_for_phrase_grounding_folder = '../data/dataset_for_phrase_grounding/'

In [10]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(BCU_database_folder + 'bcu_caption.db')
Photo = pd.read_sql_query("SELECT * from Photo", con)

# Verify that result of SQL query is stored in the dataframe
print(Photo.keys())
print('original number of picture :',len(Photo))

Index(['id', 'bcu_id', 'title', 'title_en', 'caption', 'year_from', 'year_to',
       'photographer_name', 'fund_name', 'locality', 'theme', 'filename',
       'title_en_preprocessed', 'caption_preprocessed'],
      dtype='object')
original number of picture : 2216


In [11]:
Photo = Photo.loc[Photo.filename.notnull() & (Photo.caption_preprocessed.notna() & Photo.title_en_preprocessed.notna())]
len(Photo)

1500

In [8]:
Photo.sample(100).sort_values(by='bcu_id', ascending=True)

,id,bcu_id,title,title_en,caption,year_from,year_to,photographer_name,fund_name,locality,theme,filename,title_en_preprocessed,caption_preprocessed
6,7,ALNU_00052,"Deux femmes, une en costume régional","Two women, one in regional costume",women standing in front of a church.,1885,1900,"Nussbaumer, Aloïs","Nussbaumer, Aloïs",None,Question de genre - représentations du féminin,ALNU_00052_2k_324w.jpg,"two women, one in regional costume",women standing in front of a church.
10,11,ALNU_00109,Train en arrêt à la gare de Flamatt lors de tr...,Train stop at Flamatt station during sanitatio...,steam locomotive pulling a passenger train.,1900,1900,"Nussbaumer, Aloïs","Nussbaumer, Aloïs",Flamatt,Question de la modernité - transports,ALNU_00109_2k_324w.jpg,train stop at flamatt station during sanitatio...,steam locomotive pulling a passenger train.
208,209,BERA_01689,Fribourg depuis les Grand-Places,Fribourg from the Grand-Places,tourist attraction in the fog.,1946,1946,"Rast, Benedikt","Rast, Benedikt",Fribourg,None,BERA_01689_2k_324w.jpg,from the grand-places,tourist attraction in the fog.
213,214,BERA_02514,"Pont du Diable, Loèche (Valais)","Pont du Diable, Loèche (Valais)","tourist attraction in the past, history.",1945,1955,"Rast, Benedikt","Rast, Benedikt",None,None,BERA_02514_2k_324w.jpg,"pont du diable, loèche (valais)","tourist attraction in the past, history."
230,231,BERA_03743,"Enfants (filles) en récréation, couvent des Ur...","Children (girls) in recreation, Ursulines conv...",a group of children playing in the courtyard o...,1945,1960,"Rast, Benedikt","Rast, Benedikt",Fribourg,None,BERA_03743_2k_324w.jpg,"children (girls) in recreation, ursulines conv...",a group of children playing in the courtyard o...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,2085,PRMA_00216,Salle de cours de scultpure au Technicum canto...,"Scultpure classroom at the Cantonal Technicum,...",painting artist at work in the studio.,1903,1903,"Macherel, Prosper","Macherel, Prosper",Fribourg,None,PRMA_00216_2k_324w.jpg,"scultpure classroom at the cantonal technicum,",painting artist at work in the studio.
2093,2094,PRMA_00305,"Groupe de blanchisseuses, Fribourg","Group of launderers, Fribourg",a group of women and children in a laundry room.,1900,1900,"Macherel, Prosper","Macherel, Prosper",Fribourg,Question de genre - représentations du féminin,PRMA_00305_2k_324w.jpg,"group of launderers,",a group of women and children in a laundry room.
2125,2126,PRMA_00515,"Maternité-pédiatrie, [Fribourg]","Kindergarten-pediatrics, [Fribourg]",nurses working in a hospital.,1920,1920,"Macherel, Prosper","Macherel, Prosper",Fribourg,None,PRMA_00515_2k_324w.jpg,"kindergarten-pediatrics,",nurses working in a hospital.
2149,2150,PRMA_00688,"Construction du marché couvert, Quartier de Pé...","Construction of the covered market, Periolles ...",tourist attraction in the 1930s.,1930,1930,"Macherel, Prosper","Macherel, Prosper",Fribourg,Question économique,PRMA_00688_2k_324w.jpg,"construction of the covered market, periolles ...",tourist attraction in the 1930s.


In [12]:
dataset = []
for id, row in Photo.iterrows():
    elem = dict()
    elem['image_id'] = row['id']
    elem['caption'] = dict()
    try :
        elem['caption']['raw'] = row['caption']
    except:
        elem['caption']['raw'] = ''
    try :
        elem['caption']['preprocessed'] = row['caption_preprocessed'].lower()
    except:
        elem['caption']['preprocessed'] = ''
    elem['title'] = dict()
    try :
        elem['title']['raw'] = row['title_en']
    except:
        elem['title']['raw'] = ''
    try :
        elem['title']['preprocessed'] = row['title_en_preprocessed'].lower()
    except:
        elem['title']['preprocessed'] = ''

    elem['filename'] = row['filename']
    dataset.append(elem)

    # img = cv2.imread('./data/03_resized/' + row['filename'])
    # plt.imshow(cv2.bitwise_not(img))
    # plt.show()

In [18]:
pickle.dump(dataset,open(dataset_for_phrase_grounding_folder + 'dataset.p','wb'))

for elem in dataset:
    shutil.copyfile(BCU_database_folder + '03_resized/' + elem['filename'], dataset_for_phrase_grounding_folder + 'img/' +  elem['filename'])
shutil.make_archive('../data/dataset_for_phrase_grounding', 'zip', dataset_for_phrase_grounding_folder)

'/home/tgieruc/Heritage-in-the-digital-age/data/dataset_for_phrase_grounding.zip'